## Transformation doc;variant -> patient level

In [ ]:
####### 
df_doc["avg_sentence_len_words"] = df_doc["CLEAN_FR_SPLIT"].apply(avg_sentence_length)

df_doc["hpo_per_sentence"] = df_doc.apply(
    lambda r: r["n_hpo"] / r["n_sentences"] if r["n_sentences"] > 0 else 0,
    axis=1
)

df_doc["hpo_per_1k_words"] = df_doc.apply(
    lambda r: r["n_hpo"] / r["n_words"] * 1000 if r["n_words"] > 0 else 0,
    axis=1
)


patient_agg = (
    df_doc.groupby("PATIENT_ID")
          .agg({
              "DOCUMENT_ID": "nunique",
              "CREATED_AT": ["min", "max"],
              "n_words": "sum",
              "n_sentences": "sum",
              "n_hpo": "sum",
              "n_variants": "first",
          })
)

patient_agg.columns = [
    "n_docs", "t_start", "t_end",
    "n_words_total", "n_sent_total",
    "n_hpo_total", "n_variants"
]

patient_agg["span_days"] = (patient_agg["t_end"] - patient_agg["t_start"]).dt.days



###### 
from collections.abc import Iterable

def flatten_hpo(series, unique: bool = False):
    """
    Prend une Series de listes d'HPO codes et la "flatten".
    
    - series : pd.Series où chaque élément est une liste/iterable de HPO codes
    - unique : si True, renvoie une liste triée de codes uniques
    """
    # On accepte list/tuple/set, on ignore le reste
    codes = [
        code
        for lst in series
        if isinstance(lst, Iterable) and not isinstance(lst, (str, bytes))
        for code in lst
    ]
    
    if unique:
        return sorted(set(codes))
    return codes


# Groupby sur les listes HPO par patient
grouped = df_doc.groupby("PATIENT_ID")["HPO_code"]

# Liste complète (avec répétitions)
hpo_full_list = grouped.apply(lambda s: flatten_hpo(s, unique=False))

# Liste unique (sans répétitions, triée)
hpo_unique_list = grouped.apply(lambda s: flatten_hpo(s, unique=True))

# Injection dans patient_agg (en s'assurant que l'index est PATIENT_ID)
patient_agg = patient_agg.join(
    pd.DataFrame({
        "HPO_full_list": hpo_full_list,
        "HPO_unique_list": hpo_unique_list,
    })
)

# Comptages avec .str.len() (plus lisible que apply(len))
patient_agg["n_hpo_full_list"] = patient_agg["HPO_full_list"].str.len()
patient_agg["n_hpo_unique"] = patient_agg["HPO_unique_list"].str.len()

# HPO (toutes occurrences) par document
patient_agg["hpo_total_per_doc"] = (
    patient_agg["n_hpo_full_list"] / patient_agg["n_docs"].replace(0, np.nan)
)

# HPO uniques par document
patient_agg["hpo_unique_per_doc"] = (
    patient_agg["n_hpo_unique"] / patient_agg["n_docs"].replace(0, np.nan)
)

# HPO (toutes occurrences) par phrase
patient_agg["hpo_per_sentence"] = (
    patient_agg["n_hpo_full_list"] /
    patient_agg["n_sent_total"].replace(0, np.nan)
)

# HPO (toutes occurrences) pour 1000 mots
patient_agg["hpo_per_1k_words"] = (
    patient_agg["n_hpo_full_list"] /
    patient_agg["n_words_total"].replace(0, np.nan) * 1000
)


##### 
def variant_status(n):
    n = 0 if pd.isna(n) else int(n)
    if n == 0:
        return "none"
    elif n == 1:
        return "mono"
    else:
        return "poly"

patient_agg["variant_status"] = patient_agg["n_variants"].apply(variant_status)

df_patient = patient_agg.reset_index()a